# GrainBot: A toolkit featured Segmentation, Measurement, and Statistic analysis

## Loading your Google drive and importing code from Github

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
path="/content/drive/MyDrive/GrainBot_validation"
os.chdir(path)

Mounted at /content/drive


In [2]:
!git clone https://github.com/yalan-z/GrainBot-V2.git

fatal: destination path 'GrainBot-V2' already exists and is not an empty directory.


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/GrainBot_validation/GrainBot-V2')
sys.path.append('/content/drive/MyDrive/GrainBot_validation/GrainBot-V2/GBGchar')
from importlib.machinery import SourceFileLoader
import postprocess_special as process
import mask_operation as mask_op
import AFMOpr
import MaskOpr
import GrainOpr
import StatisticM
##Install the following packages using pip first
import matplotlib.pyplot as plt
import random
import cv2
import numpy as np
import pandas as pd
segment=SourceFileLoader("ImageSeg", "/content/drive/MyDrive/GrainBot_validation/GrainBot-V2/ImageSeg.py").load_module()

## Loading SPM images and run segmentation process

In [5]:
##Make sure there are only .png images need segmentation in this path
##Remove all the outputs before rerunning the code

imgpath='../GrainBot_validation/dataset'
wei_path='/content/drive/MyDrive/GrainBot_validation/GrainBot-V2/params/unet.pth'
Dataset=segment.validation(wei_path)
Dataset.valid(imgpath)
##Successfully segmented images could be found in the same path


/content/drive/MyDrive/GrainBot_validation/GrainBot-V2/ImageSeg.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load(self.path, map_location=

successfully segmented


## Measuring microstructural parameters

In [11]:

##Input scale information of SPM images (unit:nm)
z_min=-40
z_max=40
img_width=4000
for n in Dataset.filenames:
    pddata=pd.DataFrame({'gsa':[1],'cve':[1],'rel_cve':[1],\
                     'rel_h':[1],'cca':[1],'rel_d':[1],'mean_angle':[1]})
    pd_angle=pd.DataFrame({'grain number':[1],'gsa':[1]})
    afm_img=cv2.imread(imgpath+'/'+n)
    mask_img=cv2.imread(imgpath+'/'+'seg_'+n)
    act_img=AFMOpr.AFMimg(afm_img,mask_img,z_min,z_max,img_width)
    act_img.MorphOpr()
    act_img.Labeling()
    for i in range(1,act_img.numx):

        grain=GrainOpr.Grains(i)
        grain.grainmask(act_img.labelimg,act_img.afm)
        grain.maskdil(act_img.afm)
        grain.maskregain(act_img.afm)
        grain.maskero(act_img.afm)
        grain.areacal()
        volcoe=(act_img.lmax-act_img.lmin)/255*act_img.pixelsize*act_img.pixelsize
        areacoe=act_img.pixelsize*act_img.pixelsize
        if grain.area>50:
            grain.voidcal(5)##The step of sections used for CCA or CVE calculation
            grain.anglecal(act_img.realdimg,act_img.pixelsize)

            realarea=grain.area*areacoe
            data_i=pd.DataFrame({'gsa':[realarea],'cve':[grain.vb*volcoe],\
                                 'rel_cve':[grain.rel_vb*volcoe],'rel_h':[grain.vb*volcoe/realarea],\
                                 'cca':[grain.vd*volcoe],'rel_d':[grain.vd*volcoe/realarea],\
                                 'mean_angle':[max(grain.angles)]},index=[i])
            for j in range(0,len(grain.angles)):
                angle_j=pd.DataFrame({'grain number':[i],'gsa':[grain.angles[j]]})
                pd_angle=pd.concat([pd_angle,angle_j])

            pddata=pd.concat([pddata,data_i])
        else:
            continue

    pddata.to_csv(imgpath+'/'+n+'.csv')
    pd_angle.to_csv(imgpath+'/angles_'+n+'.csv')

## Statisitcal analysis on microstructural parameters

In [ ]:

Angledata=StatisticM.StatisticAng(imgpath+'//'+'allangles_A3.0_00007.csv')
CCAdata=StatisticM.StatisticCCA(imgpath+'//'+'A3.0_00002.csv')
CVEdata=StatisticM.StatisticCVE(imgpath+'//'+'A3.0_00002.csv')
GSAdata=StatisticM.StatisticGSA(imgpath+'//'+'A3.0_00002.csv')
figure = plt.figure(figsize=(12,10),edgecolor="black",frameon=True)
plt.tight_layout()
ax=plt.subplot(2,2,1)
Angledata.histplot(1,40)
plt.text(0.55, 0.7, 'mean: ' + '%.2f' % Angledata.mu, transform=ax.transAxes, size=18)
plt.text(0.55, 0.6, 'variance: ' + '%.2f' % Angledata.sigma, transform=ax.transAxes,size=18)
plt.subplot(2,2,2)
CCAdata.histplot()
plt.subplot(2,2,3)
CVEdata.histplot()
plt.subplot(2,2,4)
GSAdata.histplot()
plt.tight_layout(pad=2,w_pad=3)
